In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf


In [ ]:
data = pd.read_csv('US_Accidents_Dec20_Updated.csv')

In [ ]:
len(data.columns)

In [ ]:
data.dtypes

In [ ]:
def accidents_ETL(accidents):
    #drop unnecessary columns
    dropthese = ['ID', 'End_Lat', 'End_Lng', 'End_Time', 'Description', 'Distance(mi)', 'Number', 'Street', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp', 'Start_Lat', 'Start_Lng', 'Nautical_Twilight', 'Astronomical_Twilight', 'Wind_Chill(F)', 'Amenity', 'Sunrise_Sunset']
    accidents = accidents.drop(columns = dropthese, axis = 1)

    #-------------------------------------------------------#
    #-------------------------------------------------------#
 
    #TWILIGHT:

    #Remname Civil Twilight
    accidents = accidents.rename(columns={'Civil_Twilight': 'Day/Night'})

    #-------------------------------------------------------#
    #-------------------------------------------------------#
    #START TIME: 

    #modify Start_Time to datetime so we can extract year, month, day, hour
    accidents['Start_Time'] = pd.to_datetime(accidents['Start_Time'])

    #extracting year, month, day, hour
    accidents['Year'] = pd.DatetimeIndex(accidents['Start_Time']).year
    accidents['Month'] = pd.DatetimeIndex(accidents['Start_Time']).month
    accidents['Day'] = pd.DatetimeIndex(accidents['Start_Time']).day
    accidents['Hour'] = pd.DatetimeIndex(accidents['Start_Time']).hour

    #drop unnecessary Start_Time column
    accidents = accidents.drop(['Start_Time'], axis=1)

    #-------------------------------------------------------#
    #-------------------------------------------------------#
 
    #SIDE: 
            
    #Changing Side to R=0 and L=1 and making it integers for ML purposes
    accidents['Side'].replace('R', '0', inplace=True)
    accidents['Side'].replace('L', '1', inplace=True)
    accidents['Side'] = np.where((accidents['Side'] == " "), '0', accidents['Side'])
    accidents['Side'] = pd.to_numeric(accidents['Side'], downcast='integer')

    #-------------------------------------------------------#
    #-------------------------------------------------------#
    #CITY AND COUNTY:

    #Combining City/State and County/State to prevent wrong city or county aggregation
    accidents['City_State'] = accidents['City'].astype(str) + ',' + accidents['State'].astype(str)
    accidents['County_State'] = accidents['County'].astype(str) + ',' + accidents['State'].astype(str)
    accidents['City'] = accidents['City_State']
    accidents['County'] = accidents['County_State']
    accidents = accidents.drop(['City_State', 'County_State'], axis=1)

    #-------------------------------------------------------#
    #-------------------------------------------------------#
    
    #WIND DIRECTIONS: 

    #Correcting Wind_Speed(mph) NaN to 0 because if there was wind it would have likely been recorded. This is run with Wind Direction due to Direction NaN processing requiring having Wind_Speed NaN set to 0 first. 
    accidents['Wind_Speed(mph)'] = accidents['Wind_Speed(mph)'].fillna(0.0)

    #Replacing wind directions to cardinal 8 directions
    accidents['Wind_Direction'].replace('CALM', 'Calm', inplace=True)
    accidents['Wind_Direction'].replace(['ENE', 'NNE'], 'NE', inplace=True)
    accidents['Wind_Direction'].replace(['ESE', 'SSE'], 'SE', inplace=True)
    accidents['Wind_Direction'].replace(['WNW', 'NNW'], 'NW', inplace=True)
    accidents['Wind_Direction'].replace(['WSW', 'SSW'], 'SW', inplace=True)
    accidents['Wind_Direction'].replace('North', 'N', inplace=True)
    accidents['Wind_Direction'].replace('East', 'E', inplace=True)
    accidents['Wind_Direction'].replace('South', 'S', inplace=True)
    accidents['Wind_Direction'].replace('West', 'W', inplace=True)
    accidents['Wind_Direction'].replace('VAR', 'Variable', inplace=True)

    #Changing unknown Wind_Direction with Wind_Speed(mi)>0 to "Variable"
    accidents['Wind_Direction'] = np.where((accidents['Wind_Direction'].isnull()) & (accidents['Wind_Speed(mph)'] > 0), 'Variable', accidents.Wind_Direction)

    #Changing Wind_Direction NaN with Wind_Speed(mi) of 0 to "Calm"
    accidents['Wind_Direction'] = np.where((accidents['Wind_Direction'].isnull()) & (accidents['Wind_Speed(mph)'] == 0), 'Calm', accidents.Wind_Direction)

    #-------------------------------------------------------#
    #-------------------------------------------------------#
 
    #WIND SPEED:

    #eliminating Wind_Speed outliers
    high_wind = accidents[accidents['Wind_Speed(mph)'] > 100]
    accidents = accidents.drop(high_wind.index)

    #Binning Wind_Speed(mph)
    bins = [-1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 101]

    #create labels for the bins
    group_labels = ["0-9", "10-19", "20-29", "30-39", "40-49", "50-59", "60-69", "70-79", "80-89", "90-100",] 

    #slice the Humidity data and place into bins
    accidents['Wind_Speed(mph)'] = pd.cut(accidents['Wind_Speed(mph)'], bins, labels=group_labels)

    #casting Temperature as object for encoding purposes
    accidents['Wind_Speed(mph)'] = accidents['Wind_Speed(mph)'].astype(object)

    #-------------------------------------------------------#
    #-------------------------------------------------------#
 
    #HUMIDITY:

    #Chaning Humidity NaN to "0"
    accidents['Humidity(%)'] = accidents['Humidity(%)'].fillna('101')

    #change column to numerical
    accidents['Humidity(%)'] = accidents['Humidity(%)'].astype(int)

    #Binning Humidity
    bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 101]

    #create labels for the bins
    group_labels = ["0-9%", "10-19%", "20-29%", "30-39%", "40-49%", "50-59%", "60-69%", "70-79%", "80-89%", "90-100%", "Unknown"] 

    #slice the Humidity data and place into bins
    accidents['Humidity(%)'] = pd.cut(accidents['Humidity(%)'], bins, labels=group_labels)

    #casting Humidity as object for encoding purposes
    accidents['Humidity(%)'] = accidents['Humidity(%)'].astype(object)

    #-------------------------------------------------------#
    #-------------------------------------------------------#
    
    #DAY/NIGHT:

    #Dropping Day/Night NaN values - because daylight hours change and cannot be reliably determined by time and it is 110 values.
    noDayNight = accidents[accidents['Day/Night'].isnull()]
    accidents = accidents.drop(noDayNight.index)

    #-------------------------------------------------------#
    #-------------------------------------------------------#
 
    #ZIPCODE:

    #Dropping Zipcodes because of an extremely large number of variables and no extra value added over city, county, state.
    accidents.drop('Zipcode', axis=1, inplace=True)

    #-------------------------------------------------------#
    #-------------------------------------------------------#
 
    #PRECIPITATION: 

    #Correcting Precipitation(in) NaN to 0 because if there was precipitation it would have likely been recorded. 
    accidents['Precipitation(in)'] = accidents['Precipitation(in)'].fillna(0.00)

    #eliminating Precipitation(in) outliers
    high_rain = accidents[accidents['Precipitation(in)'] > 13]
    accidents = accidents.drop(high_rain.index)

    # Precipitation Binning
    bins = [-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

    #create labels for the bins
    group_labels = ["0-1", "1-2", "2-3", "3-4", "4-5", "5-6", "6-7", "7-8", "8-9", "9-10", "10-11"] 

    #slice the Humidity data and place into bins
    accidents['Precipitation(in)'] = pd.cut(accidents['Precipitation(in)'], bins, labels=group_labels)

    #casting Temperature as object for encoding purposes
    accidents['Precipitation(in)'] = accidents['Precipitation(in)'].astype(object)

    #-------------------------------------------------------#
    #-------------------------------------------------------#
   
    #TEMPERATURE:

    #Changing Temperature NaN to "Unknown"
    accidents['Temperature(F)'] = accidents['Temperature(F)'].fillna('500')

    #converting Temperature(F) to float
    accidents['Temperature(F)'] = accidents['Temperature(F)'].astype(float)

    #dropping values below -30F
    dropping = accidents[accidents['Temperature(F)'] < -30]
    accidents.drop(dropping.index, inplace=True)

    #dropping over 120F
    dropping2 =  accidents[(accidents['Temperature(F)'] > 120) & (accidents['Temperature(F)'] <500)]
    accidents.drop(dropping2.index, inplace=True)

    #Binning Temperature
    bins = [-30, -20, -10, 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 121, 500]
    #create labels for the bins
    group_labels = ["-30F:-20F", "-19F:-10F", "-10F:0F", "0F-9F", "10F:19F", "20F:29F", "30F:39F", "40F:49F", "50F:59F", "60F:69F", "70F:79F", "80F:89F", "90F:99F", "100F:109F", "110F:120F", "Unknown"] 

    #slice the Humidity data and place into bins
    accidents['Temperature(F)'] = pd.cut(accidents['Temperature(F)'], bins, labels=group_labels)

    #casting Temperature as object for encoding purposes
    accidents['Temperature(F)'] = accidents['Temperature(F)'].astype(object)

    #-------------------------------------------------------#
    #-------------------------------------------------------#
 
    #PRESSURE: 

    #Changing Pressure NaN to "Unknown"
    accidents['Pressure(in)'] = accidents['Pressure(in)'].fillna('500')

    #Casting Pressure as float to ensure dropping works.
    accidents['Pressure(in)'] = accidents['Pressure(in)'].astype(float) 

    #dropping Pressure(in) values over 32in and removing a faulty row
    dropping3 = accidents[(accidents['Pressure(in)'] >32) & (accidents['Pressure(in)'] <500)]
    accidents.drop(dropping3.index, inplace=True)
    accidents.drop(index=879551, inplace=True)

    #Binning Pressure(in)
    bins = [-1, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 35, 500]

    #create labels for the bins
    group_labels = ["0-2", "2-4", "4-6", "6-8", "8-10", "10-12", "12-14", "14-16", "16-18", "18-20", "20-22", "22-24", "24-26", "26-28", "28-30", "30-32", "32-34", "Unknown"] 

    #slice the Humidity data and place into bins
    accidents['Pressure(in)'] = pd.cut(accidents['Pressure(in)'], bins, labels=group_labels)

    #casting Temperature as object for encoding purposes
    accidents['Pressure(in)'] = accidents['Pressure(in)'].astype(object) 

    #-------------------------------------------------------#
    #-------------------------------------------------------#
 
    #VISIBILITY:

    #Change Visibilty(mi) NaN values to the median of 10.0
    accidents['Visibility(mi)'] = accidents['Visibility(mi)'].fillna(10.00)

    #Change anything above 10mi to 10mi because that is considered max visibility and nothing beyond that matters to driving.
    accidents['Visibility(mi)'] = np.where((accidents['Visibility(mi)'] > 10) , 10, accidents['Visibility(mi)'])

    #Binning Visibility(mi)
    bins = [-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11]

    #create labels for the bins
    group_labels = ["0-1", "1-2", "2-3", "3-4", "4-5", "5-6", "6-7", "7-8", "8-9", "9-10"] 

    #slice the Humidity data and place into bins
    accidents['Visibility(mi)'] = pd.cut(accidents['Visibility(mi)'], bins, labels=group_labels)

    #casting Temperature as object for encoding purposes
    accidents['Visibility(mi)'] = accidents['Visibility(mi)'].astype(object)

    #-------------------------------------------------------#
    #-------------------------------------------------------#
 
    #WEATHER CONDITION:

    #Consolidating Variables

    #Clear
    accidents['Weather_Condition'].replace('Clear', 'Fair', inplace=True)
    accidents['Weather_Condition'].replace('N/A Precipitation', 'Fair', inplace=True)

    #Chaning Weather_Condition NaN to "Unknown"
    accidents['Weather_Condition'] = accidents['Weather_Condition'].fillna('Fair')

    #Cloudy
    accidents['Weather_Condition'].replace('Mostly Cloudy', 'Mostly_Cloudy', inplace=True)
    accidents['Weather_Condition'].replace('Partly Cloudy', 'Partly_Cloudy', inplace=True)
    accidents['Weather_Condition'].replace('Overcast', 'Mostly_Cloudy', inplace=True)
    accidents['Weather_Condition'].replace('Scattered Clouds', 'Partly_Cloudy', inplace=True)

    #Rain
    accidents['Weather_Condition'].replace('Light Rain', 'Light_Rain', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Rain', 'Heavy_Rain', inplace=True)
    accidents['Weather_Condition'].replace('Light Rain Shower', 'Light_Rain', inplace=True)
    accidents['Weather_Condition'].replace('Light Rain Showers', 'Light_Rain', inplace=True)
    accidents['Weather_Condition'].replace('Light Drizzle', 'Light_Rain', inplace=True)
    accidents['Weather_Condition'].replace('Light Freezing Drizzle', 'Light_Rain', inplace=True)
    accidents['Weather_Condition'].replace('Light Freezing Rain', 'Light_Rain', inplace=True)
    accidents['Weather_Condition'].replace('Drizzle', 'Light_Rain', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Drizzle', 'Rain', inplace=True)
    accidents['Weather_Condition'].replace('Showers in the Vicinity', 'Rain', inplace=True)
    accidents['Weather_Condition'].replace('Rain Showers', 'Rain', inplace=True)
    accidents['Weather_Condition'].replace('Rain Shower', 'Rain', inplace=True)
    accidents['Weather_Condition'].replace('Freezing Rain', 'Rain', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Rain Shower', 'Heavy_Rain', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Freezing Rain', 'Heavy_Rain', inplace=True)
    accidents['Weather_Condition'].replace('Freezing Drizzle', 'Rain', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Rain Showers', 'Heavy_Rain', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Freezing Drizzle', 'Rain', inplace=True)

    #Fog
    accidents['Weather_Condition'].replace('Patches of Fog', 'Fog', inplace=True)
    accidents['Weather_Condition'].replace('Light Freezing Fog', 'Fog', inplace=True)
    accidents['Weather_Condition'].replace('Partial Fog', 'Fog', inplace=True)
    accidents['Weather_Condition'].replace('Light Fog', 'Fog', inplace=True)
    accidents['Weather_Condition'].replace('Shallow Fog', 'Shallow_Fog', inplace=True)

    #Smoke/Haze
    accidents['Weather_Condition'].replace('Smoke', 'Smoke/Haze', inplace=True)
    accidents['Weather_Condition'].replace('Haze', 'Smoke/Haze', inplace=True)
    accidents['Weather_Condition'].replace('Light Haze', 'Smoke/Haze', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Smoke', 'Smoke/Haze', inplace=True)

    #Thunderstorms
    accidents['Weather_Condition'].replace('T-Storm', 'Thunderstorm', inplace=True)
    accidents['Weather_Condition'].replace('Light Thunderstorms and Rain', 'Thunderstorm', inplace=True)
    accidents['Weather_Condition'].replace('Thunder in the Vicinity', 'Thunder', inplace=True)
    accidents['Weather_Condition'].replace('Light Rain with Thunder', 'Thunderstorm', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Thunderstorms and Rain', 'Thunderstorm', inplace=True)
    accidents['Weather_Condition'].replace('Heavy T-Storm', 'Thunderstorm', inplace=True)
    accidents['Weather_Condition'].replace('Thunderstorms and Rain', 'Thunderstorm', inplace=True)
    accidents['Weather_Condition'].replace('Light Thunderstorms and Snow', 'Thunderstorms/Snow', inplace=True)
    accidents['Weather_Condition'].replace('Thunderstorms and Snow', 'Thunderstorms/Snow', inplace=True)
    accidents['Weather_Condition'].replace('Light Thunderstorm', 'Thunderstorm', inplace=True)

    #Snow
    accidents['Weather_Condition'].replace('Light Snow', 'Light_Snow', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Snow', 'Heavy_Snow', inplace=True)
    accidents['Weather_Condition'].replace('Light Snow Showers', 'Snow', inplace=True)
    accidents['Weather_Condition'].replace('Snow Grains', 'Snow', inplace=True)
    accidents['Weather_Condition'].replace('Light Blowing Snow', 'Snow/Windy', inplace=True)
    accidents['Weather_Condition'].replace('Light Snow Grains', 'Snow', inplace=True)
    accidents['Weather_Condition'].replace('Low Drifting Snow', 'Snow', inplace=True)
    accidents['Weather_Condition'].replace('Snow Showers', 'Snow', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Blowing Snow', 'Heavy Snow/Windy', inplace=True)
    accidents['Weather_Condition'].replace('Light Snow Shower', 'Snow', inplace=True)
    accidents['Weather_Condition'].replace('Drifting Snow', 'Snow/Windy', inplace=True)

    #Hail
    accidents['Weather_Condition'].replace('Light Ice Pellets', 'Hail', inplace=True)
    accidents['Weather_Condition'].replace('Ice Pellets', 'Hail', inplace=True)
    accidents['Weather_Condition'].replace('Small Hail', 'Hail', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Ice Pellets', 'Hail', inplace=True)
    accidents['Weather_Condition'].replace('Light Hail', 'Hail', inplace=True)

    #Sleet
    accidents['Weather_Condition'].replace('Light Sleet', 'Sleet', inplace=True)
    accidents['Weather_Condition'].replace('Heavy Sleet', 'Sleet', inplace=True)

    #Dust
    accidents['Weather_Condition'].replace('Sand', 'Dust', inplace=True)
    accidents['Weather_Condition'].replace('Widespread Dust', 'Dust', inplace=True)

    #Tornado
    accidents['Weather_Condition'].replace('Tornado', 'Funnel_Cloud', inplace=True) 
    accidents['Weather_Condition'].replace('Funnel Cloud', 'Funnel_Cloud', inplace=True)    

    #Dust Whirls
    accidents['Weather_Condition'].replace('Dust Whirls', 'Dust_Whirls', inplace=True)

    #Wintry Mix
    accidents['Weather_Condition'].replace('Wintry Mix', 'Wintry_Mix', inplace=True) 

    #Wind
    accidents = accidents.assign(Weather_Condition_Wind = 0)

    #Volcanic Ash
    accidents['Weather_Condition'].replace('Volcanic Ash', 'Volcanic_Ash', inplace=True)


    #-------------------------------------------------------#
    #-------------------------------------------------------#

    accidents = pd.get_dummies(accidents, columns = ['Weather_Condition'])

    for index, row in accidents.iterrows():
        if accidents.loc[index, 'Weather_Condition_Smoke / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Wind'] = 1
            accidents.loc[index, 'Weather_Condition_Smoke/Haze'] = 1
        
        elif accidents.loc[index, 'Weather_Condition_Blowing Dust'] == 1:
            accidents.loc[index, 'Weather_Condition_Dust'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Blowing Dust / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Dust'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Blowing Sand'] == 1:
            accidents.loc[index, 'Weather_Condition_Dust'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1
        
        elif accidents.loc[index, 'Weather_Condition_Blowing Snow'] == 1:
            accidents.loc[index, 'Weather_Condition_Snow'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Blowing Snow / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Snow'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Cloudy / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Cloudy'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Drizzle / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Light_Rain'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Drizzle and Fog'] == 1:
            accidents.loc[index, 'Weather_Condition_Light_Rain'] = 1
            accidents.loc[index, 'Weather_Condition_Fog'] = 1

        elif accidents.loc[index, 'Weather_Condition_Fair / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Fair'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Fog / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Fog'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Freezing Rain / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Wind'] = 1
            accidents.loc[index, 'Weather_Condition_Rain'] = 1

        elif accidents.loc[index, 'Weather_Condition_Haze / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Smoke/Haze'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1
        
        elif accidents.loc[index, 'Weather_Condition_Heavy Rain / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Heavy_Rain'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1
        
        elif accidents.loc[index, 'Weather_Condition_Heavy Snow / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Heavy_Snow'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1
                
        elif accidents.loc[index, 'Weather_Condition_Heavy Snow with Thunder'] == 1:
            accidents.loc[index, 'Weather_Condition_Heavy_Snow'] = 1
            accidents.loc[index, 'Weather_Condition_Thunder'] = 1

        elif accidents.loc[index, 'Weather_Condition_Heavy Snow/Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Heavy_Snow'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Heavy T-Storm / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Wind'] = 1
            accidents.loc[index, 'Weather_Condition_Thunderstorm'] = 1

        elif accidents.loc[index, 'Weather_Condition_Heavy Thunderstorms and Snow'] == 1:
            accidents.loc[index, 'Weather_Condition_Snow'] = 1
            accidents.loc[index, 'Weather_Condition_Thunderstorm'] = 1

        elif accidents.loc[index, 'Weather_Condition_Heavy Thunderstorms with Small Hail'] == 1:
            accidents.loc[index, 'Weather_Condition_Thunderstorm'] = 1
            accidents.loc[index, 'Weather_Condition_Hail'] = 1

        elif accidents.loc[index, 'Weather_Condition_Light Drizzle / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Light_Rain'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Light Freezing Rain / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Wind'] = 1
            accidents.loc[index, 'Weather_Condition_Light_Rain'] = 1
        
        elif accidents.loc[index, 'Weather_Condition_Light Rain / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Light_Rain'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1
        
        elif accidents.loc[index, 'Weather_Condition_Light Rain Shower / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Light_Rain'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1
        
        elif accidents.loc[index, 'Weather_Condition_Light Sleet / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Sleet'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1
                        
        elif accidents.loc[index, 'Weather_Condition_Light Snow / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Light_Snow'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1
                        
        elif accidents.loc[index, 'Weather_Condition_Light Snow and Sleet'] == 1:
            accidents.loc[index, 'Weather_Condition_Light_Snow'] = 1
            accidents.loc[index, 'Weather_Condition_Sleet'] = 1
                        
        elif accidents.loc[index, 'Weather_Condition_Light Snow and Sleet / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Light_Snow'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1
                        
        elif accidents.loc[index, 'Weather_Condition_Light Snow with Thunder'] == 1:
            accidents.loc[index, 'Weather_Condition_Light_Snow'] = 1
            accidents.loc[index, 'Weather_Condition_Thunder'] = 1

        elif accidents.loc[index, 'Weather_Condition_Mist / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Mist'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Mostly Cloudy / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Mostly_Cloudy'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Partly Cloudy / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Partly_Cloudy'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Patches of Fog / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Fog'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Rain / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Rain'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Rain and Sleet'] == 1:
            accidents.loc[index, 'Weather_Condition_Rain'] = 1
            accidents.loc[index, 'Weather_Condition_Sleet'] = 1

        elif accidents.loc[index, 'Weather_Condition_Sand / Dust Whirls Nearby'] == 1:
            accidents.loc[index, 'Weather_Condition_Sand'] = 1
            accidents.loc[index, 'Weather_Condition_Dust_Whirls'] = 1

        elif accidents.loc[index, 'Weather_Condition_Sand / Dust Whirlwinds'] == 1:
            accidents.loc[index, 'Weather_Condition_Sand'] = 1
            accidents.loc[index, 'Weather_Condition_Dust_Whirls'] = 1

        elif accidents.loc[index, 'Weather_Condition_Sleet / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Sleet'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Smoke / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Smoke/Haze'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Snow / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Snow'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1
                
        elif accidents.loc[index, 'Weather_Condition_Snow and Sleet'] == 1:
            accidents.loc[index, 'Weather_Condition_Snow'] = 1
            accidents.loc[index, 'Weather_Condition_Sleet'] = 1

        elif accidents.loc[index, 'Weather_Condition_Snow and Sleet / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Snow'] = 1
            accidents.loc[index, 'Weather_Condition_Sleet'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Snow/Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Snow'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Squalls / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Squalls'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_T-Storm / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Thunderstorm'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Thunder / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Thunder'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Thunder / Wintry Mix'] == 1:
            accidents.loc[index, 'Weather_Condition_Thunder'] = 1
            accidents.loc[index, 'Weather_Condition_Wintry_Mix'] = 1

        elif accidents.loc[index, 'Weather_Condition_Thunder / Wintry Mix / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Wintry_Mix'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Thunder and Hail'] == 1:
            accidents.loc[index, 'Weather_Condition_Thunder'] = 1
            accidents.loc[index, 'Weather_Condition_Hail'] = 1

        elif accidents.loc[index, 'Weather_Condition_Thunder and Hail / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Hail'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Thunderstorms/Snow'] == 1:
            accidents.loc[index, 'Weather_Condition_Thunderstorm'] = 1
            accidents.loc[index, 'Weather_Condition_Snow'] = 1

        elif accidents.loc[index, 'Weather_Condition_Widespread Dust / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Dust'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1

        elif accidents.loc[index, 'Weather_Condition_Wintry Mix / Windy'] == 1:
            accidents.loc[index, 'Weather_Condition_Wintry_Mix'] = 1
            accidents.loc[index, 'Weather_Condition_Wind'] = 1    


    #-------------------------------------------------------#
    #-------------------------------------------------------#
  
    #Set columns to drop 
    cols_to_drop = ['Weather_Condition_Smoke / Windy', 'Weather_Condition_Blowing Dust',   'Weather_Condition_Blowing Dust / Windy', 'Weather_Condition_Blowing Sand', 'Weather_Condition_Blowing Snow', 'Weather_Condition_Blowing Snow / Windy', 'Weather_Condition_Cloudy / Windy', 'Weather_Condition_Drizzle / Windy', 'Weather_Condition_Drizzle and Fog', 'Weather_Condition_Fair / Windy', 'Weather_Condition_Fog / Windy', 'Weather_Condition_Freezing Rain / Windy', 'Weather_Condition_Haze / Windy', 'Weather_Condition_Heavy Rain / Windy', 'Weather_Condition_Heavy Snow / Windy', 'Weather_Condition_Heavy Snow with Thunder', 'Weather_Condition_Heavy Snow/Windy', 'Weather_Condition_Heavy T-Storm / Windy', 'Weather_Condition_Heavy Thunderstorms and Snow', 'Weather_Condition_Heavy Thunderstorms with Small Hail', 'Weather_Condition_Light Drizzle / Windy', 'Weather_Condition_Light Freezing Rain / Windy', 'Weather_Condition_Light Rain / Windy', 'Weather_Condition_Light Rain Shower / Windy', 'Weather_Condition_Light Sleet / Windy', 'Weather_Condition_Light Snow / Windy', 'Weather_Condition_Light Snow and Sleet', 'Weather_Condition_Light Snow and Sleet / Windy', 'Weather_Condition_Light Snow with Thunder', 'Weather_Condition_Mist / Windy', 'Weather_Condition_Mostly Cloudy / Windy', 'Weather_Condition_Partly Cloudy / Windy', 'Weather_Condition_Patches of Fog / Windy', 'Weather_Condition_Rain and Sleet', 'Weather_Condition_Sand', 'Weather_Condition_Sand / Dust Whirls Nearby', 'Weather_Condition_Sand / Dust Whirlwinds', 'Weather_Condition_Sleet / Windy', 'Weather_Condition_Smoke / Windy', 'Weather_Condition_Snow / Windy', 'Weather_Condition_Snow and Sleet', 'Weather_Condition_Snow and Sleet / Windy', 'Weather_Condition_Snow/Windy', 'Weather_Condition_Squalls / Windy', 'Weather_Condition_T-Storm / Windy', 'Weather_Condition_Thunder / Windy', 'Weather_Condition_Thunder / Wintry Mix', 'Weather_Condition_Thunder / Wintry Mix / Windy', 'Weather_Condition_Thunder and Hail', 'Weather_Condition_Thunder and Hail / Windy', 'Weather_Condition_Thunderstorms/Snow', 'Weather_Condition_Widespread Dust / Windy', 'Weather_Condition_Wintry Mix / Windy', 'Weather_Condition_Rain / Windy']

    # Drop the columns
    accidents = accidents.drop(columns = cols_to_drop, axis=1)  

    #-------------------------------------------------------#
    #-------------------------------------------------------#
   
    #BOOLEAN COLUMN CONVERSIONS:

    #Converting Bool columns to integers to prepare for scaling and ML modeling
    bool_cols = accidents.dtypes[accidents.dtypes == 'bool'].index.tolist()
    bool_int = accidents[bool_cols].astype(int)
    encoding = accidents.copy()
    encoding = accidents.drop(bool_int,1)

    #encoding = encoding.merge(bool_int, left_index=True, right_index=True)
    accidents = encoding.merge(bool_int, left_index=True, right_index=True)


    return(accidents)


In [ ]:
accidents = accidents_ETL(data)
accidents

In [ ]:
    #Encoding Obj columns using OneHotEncoder

    #Identifying the 'object' columns to encode
    type_cat = accidents.dtypes[accidents.dtypes == 'object'].index.tolist()
    type_cat

    #Create a OneHotEncoder instance
    enc = OneHotEncoder(sparse=False)

    # Fit and transform the OneHotEncoder using the categorical variable list
    encode_df = pd.DataFrame(enc.fit_transform(accidents[type_cat].astype(str)))

    # Add the encoded variable names to the dataframe
    encode_df.columns = enc.get_feature_names(type_cat)
    encode_df.head()

    #Merge one-hot encoded features and drop the originals
    accidents = accidents.merge(encode_df, left_index=True, right_index=True)
    accidents = accidents.drop(type_cat,1)

In [ ]:
output_data_file = "Accidents_Preprocessed1.csv"
accidents.to_csv(output_data_file, index=False)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1)

# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Create a classifier object
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=learning_rate,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)

    # Fit the model
    classifier.fit(X_train_scaled, y_train)
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train_scaled,
            y_train)))
    print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test_scaled,
            y_test)))
    print()


In [ ]:
# Choose a learning rate and create classifier
classifier = GradientBoostingClassifier(n_estimators=20,
                                        learning_rate=0.5,
                                        max_features=5,
                                        max_depth=3,
                                        random_state=0)

# Fit the model
classifier.fit(X_train_scaled, y_train)

# Make Prediction
predictions = classifier.predict(X_test_scaled)
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).head(20)